## Prep VOR data (2013-2021) for comparison with APEX and HLS biomass predictions

#### Load required packages

In [71]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import matplotlib.pyplot as plt
from shapely.geometry import MultiPoint

#### Define inputs/outputs

In [85]:
transects_f = "C:/SPK_local/data/vectors/VOR_transects/CPER_VOR_Transects_all_2020_Oct14.shp"
vor_baseDIR = 'T:/2-Projects/AGM/Measurements - Annual Decisions/Vegetation/VOR/'
vor_juneDIR = 'June/MultiYears/2013-2021/'
vor_june_f = 'CARM_VOR_JUN_cln_attr_ALL_HiLo2021-10-01.csv'
vor_octDIR = 'October/MultiYears/2013-2021/CARM_VOR_OCT_cln_attr_FORAGE_HiLo2021-10-26/' 
vor_oct_f = 'CARM_VOR_OCT_cln_attr_ALL_HiLo2021-10-26.csv'
cper_2017_f = "C:/SPK_local/data/vectors/Pasture_Boundaries/Shapefiles/cper_pastures_2017_clip.shp"

vor_dict = {
    2014: "C:/SPK_local/data/vegetation/CARM_weekly_VOR/Weekly_VOR_2014_simp.csv",
    2015: "C:/SPK_local/data/vegetation/CARM_weekly_VOR/Weekly_VOR_2015_simp.csv",
    2016: "C:/SPK_local/data/vegetation/CARM_weekly_VOR/Weekly_VOR_2016_simp.csv",
    2017: "C:/SPK_local/data/vegetation/CARM_weekly_VOR/Weekly_VOR_2017_simp.csv",
    2018: "C:/SPK_local/data/vegetation/CARM_weekly_VOR/Weekly_VOR_2018_simp.csv",
    2019: "C:/SPK_local/data/vegetation/CARM_weekly_VOR/Weekly_VOR_2019_simp.csv",
    2020: "C:/SPK_local/data/vegetation/CARM_weekly_VOR/Weekly_VOR_2020_simp.csv",
    2021: "C:/SPK_local/data/vegetation/CARM_weekly_VOR/Weekly_VOR_2021_simp.csv",
}

outDIR = 'data/'
if not os.path.exists(outDIR):
    os.mkdir(outDIR)

#### Define additional variables and information for data cleaning and preparation

In [3]:
# list of species to remove from datasets
spp_rem = ['OPPO', 'ATCA', 'GUSA', 'EREF', 'ARFR', 'ERNA', 'KRLA', 'YUGL']

# dictionary of species names to change
spp_chg = {"ANT": "BARE", "ARME": "ARPO", "CACU": "CADU", "CAIN": "CANE", "CELA": "KRLA", "CMRI": "CRMI",
           "EBNU": "EVNU", "LYGU": "LYJU", "SIAB": "SAIB", "SIHY": "ELEL", "STCO": "HECO",
           "UNK": "UNKN", "UNKF": "UNFB", "UKNF": "UNFB", "VUO": "VUOC", "BA": "BARE", "BU": "BUDA",
           "CA": "CADU", "HE": "HECO", "VU": "VUOC", "BNO": "BOGR", "BRTI": "BRTE", "LITTT": "LITT",
           "ALIT": "ALTE", "BGOR": "BOGR", "VARE": "BARE", "ALTI": "ALTE", "AT": "ATCA", "BR": "BRTE",
           "KLI": "KRLA", "LOPP": "OPPO", "PA": "PASM", "POASM": "PASM", "SP": "SPAI", "VO": "VUOC",
           "oldforb": "FORB", "BO": "BOBU", "std": "SD", "stde": "SD", "bu": "BUDA", "vo": "VUOC", "vouc": "VUOC",
           "AFRF": "ARFR", "LIT": "LITT", "lit": "LITT"}

# species lists and build search for quantify species group
base = r'^{}'
expr = '|(?=.*{})'
c4_spp = ['BOGR', 'BUDA', 'ARPU']
c3_spp = ['PASM', 'HECO', 'CADU', 'ELEL', 'CAFI', 'DISP', 'MUTO', 'SPCR', 'SPAI', 'STCO']
ann_spp = ['VUOC', 'BRTE']
litt_spp = ['LITT']
bare_spp = ['BARE', 'DUNG', 'ROCK']
sd_spp = {'SD'}
c4_search = re.sub('\\^\\|', r'^', base.format(''.join(expr.format(w) for w in c4_spp)))
c3_search = re.sub('\\^\\|', r'^', base.format(''.join(expr.format(w) for w in c3_spp)))
ann_search = re.sub('\\^\\|', r'^', base.format(''.join(expr.format(w) for w in ann_spp)))
litt_search = re.sub('\\^\\|', r'^', base.format(''.join(expr.format(w) for w in litt_spp)))
bare_search = re.sub('\\^\\|', r'^', base.format(''.join(expr.format(w) for w in bare_spp)))
sd_search = re.sub('\\^\\|', r'^', base.format(''.join(expr.format(w) for w in sd_spp)))

# coefficients for VOR to biomass regression
coef_intcp = 32.9
coef_low = 133.6
coef_hi = 23.7

# set experimental unit to 'Plot' or 'Transect'
exp_unit = "Plot"

# set filters for VOR
max_vor = 50
max_sd = 2.5

#### Load and prepare transect data

In [79]:
# load CPER shapefile
cper = gpd.read_file(cper_2017_f)

# load transect shapefile
df_transects = gpd.read_file(transects_f)

# manually rename transect pastures for consistency
df_transects.loc[df_transects['PastureNm'] == 'NH', 'PastureNm'] = '10S'
df_transects.loc[df_transects['PastureNm'] == '7se', 'PastureNm'] = '7SE'
df_transects.loc[df_transects['PastureNm'] == '31w', 'PastureNm'] = '31W'
df_transects.loc[df_transects['PastureNm'] == '26w', 'PastureNm'] = '26W'
df_transects.loc[df_transects['PastureNm'] == '25nw', 'PastureNm'] = '25NW'
df_transects.loc[df_transects['PastureNm'] == '17s', 'PastureNm'] = '17S'
df_transects.loc[df_transects['PastureNm'] == '18s', 'PastureNm'] = '18S'
df_transects.loc[df_transects['PastureNm'] == '19n', 'PastureNm'] = '19N'
df_transects.loc[df_transects['PastureNm'] == '19n', 'PastureNm'] = '19N'
df_transects.loc[df_transects['PastureNm'] == '21n', 'PastureNm'] = '21N'
df_transects.loc[df_transects['PastureNm'] == '22SW', 'PastureNm'] = '22W'

# strip letters from transect and plot numbers
df_transects['Transect'] = df_transects['Transect'].apply(lambda x: re.sub("T", "", str(x)))
df_transects['Transect'] = df_transects['Transect'].apply(lambda x: re.sub("P", "", str(x)))

# create unique transect ID's
df_transects['Id'] = df_transects['PastureNm'] + "_" + \
                     df_transects['Plot_Num'] + "_" + df_transects['Transect'].astype('str')
df_transects['Id_plot'] = df_transects['PastureNm'] + "_" + \
                          df_transects['Plot_Num']

# create plot polygons from transects, buffered by 15 m
df_transects['geometry'] = df_transects['geometry'].apply(lambda x: x.coords[0])
df_plots = df_transects.groupby(['Id_plot'])['geometry'].apply(lambda x: MultiPoint(x.tolist()).convex_hull)
df_plots = gpd.GeoDataFrame(df_plots, geometry='geometry', crs=df_transects.crs).buffer(15)
df_transects = df_transects.groupby(['Id'])['geometry'].apply(lambda x: MultiPoint(x.tolist()).convex_hull)
df_transects = gpd.GeoDataFrame(df_transects, geometry='geometry', crs=df_plots.crs).buffer(15)

#### Load and prepare weekly VOR data

In [79]:
df_vor_2014 = pd.read_csv(vor_dict[2014])
df_vor_2015 = pd.read_csv(vor_dict[2015])
df_vor_2016 = pd.read_csv(vor_dict[2016])

df_vor_2017 = pd.read_csv(vor_dict[2017])
df_vor_2017.loc[df_vor_2017['Pasture'] == '18S_burn', 'PastureNm'] = '18S'
df_vor_2017.loc[df_vor_2017['Pasture'] == '7SE_salt', 'PastureNm'] = '7SE'
df_vor_2017.loc[df_vor_2017['Pasture'] == 'NH', 'PastureNm'] = '10S'

df_vor_2018 = pd.read_csv(vor_dict[2018])
df_vor_2018.loc[df_vor_2018['Plot'] == '4_burn', 'Plot'] = '4'
df_vor_2018.loc[df_vor_2018['Pasture'] == '20NW_saltflat', 'PastureNm'] = '20NW'

df_vor_2019 = pd.read_csv(vor_dict[2019], engine='python', encoding='cp1252') # needed due to encoding error?
df_vor_2019.loc[df_vor_2019['Plot'] == '4_burn', 'Plot'] = '4'
df_vor_2019 = df_vor_2019[df_vor_2019['Low'] != 'dung']
df_vor_2019['Low'] = df_vor_2019['Low'].astype('int64')

df_vor_2020 = pd.read_csv(vor_dict[2020], engine='python')
df_vor_2020.loc[df_vor_2020['Pasture'] == 'NH', 'Pasture'] = '10S'
df_vor_2020.loc[df_vor_2020['Plot'] == '4_burn', 'Plot'] = '4'
df_vor_2020 = df_vor_2020[df_vor_2020['Low'] != 'dung']
df_vor_2020['Low'] = df_vor_2020['Low'].astype('int64')

df_vor_2021 = pd.read_csv(vor_dict[2021], engine='python')
df_vor_2021.loc[df_vor_2021['Pasture'] == 'NH', 'Pasture'] = '10S'

df_vor = pd.concat([df_vor_2014, df_vor_2015, df_vor_2016, df_vor_2017, 
                    df_vor_2018, df_vor_2019, df_vor_2020, df_vor_2021])
df_vor = df_vor[~(df_vor.Low > df_vor.High)]  # remove nonsensical data
df_vor['Spp'] = [re.sub(' ', '', str(x)) for x in df_vor['Spp']]  # remove any spaces in species names
for name_old, name_new in spp_chg.items():
    df_vor['Spp'] = df_vor['Spp'].replace(name_old, name_new)  # replace erroneous species names
df_vor['Spp'] = df_vor['Spp'].str.upper()  # convert all species names to upper case
df_vor = df_vor[~(df_vor['Spp'].isin(spp_rem))]  # remove non-herbaceous species
df_vor = df_vor[~(df_vor['Comments'].str.contains("emove", na=False))]
df_vor['Date'] = pd.to_datetime(df_vor['Date'])
df_vor['Year'] = df_vor['Date'].dt.year
df_vor['bm'] = df_vor['Low'] * coef_low + df_vor['High'] * coef_hi + coef_intcp
df_vor.loc[df_vor['G_B'] == 'G', 'G_B'] = 'g'
df_vor.loc[df_vor['G_B'] == 'B', 'G_B'] = 'b'
df_vor['Plot'] = "P" + df_vor['Plot'].astype('str')

C:\Users\sean.kearney\.conda\envs\py_geo_env\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


#### Load and prepare June VOR data

In [58]:
df_vor_june = pd.read_csv(os.path.join(vor_baseDIR, vor_juneDIR, vor_june_f),
                          usecols=np.arange(0, 15), engine='python')
df_vor_june.loc[df_vor_june['SamplingDate'] == '11-Jun', 
                'SamplingDate'] = '6/11/' + str(df_vor_june.loc[df_vor_june['SamplingDate'] == '11-Jun', 'Year'].astype('int').unique().item())
df_vor_june = df_vor_june[~df_vor_june.isnull().all(axis=1)]
df_vor_june['Date'] = ['6/12/' + str(int(i)) for i in df_vor_june['Year']]
df_vor_june['Date'] = pd.to_datetime(df_vor_june['Date'], format='%m/%d/%Y')
df_vor_june.loc[df_vor_june['SamplingDate'].notnull(), 'Date'] = pd.to_datetime(df_vor_june.loc[df_vor_june['SamplingDate'].notnull(), 'SamplingDate'])
df_vor_june.loc[df_vor_june['Pasture'] == 'NH', 'Pasture'] = '10S'
df_vor_june['East_SppLo'] = [re.sub(' ', '', str(x)) for x in
                            df_vor_june['East_SppLo']]  # remove any spaces in species names
df_vor_june['West_SppLo'] = [re.sub(' ', '', str(x)) for x in
                            df_vor_june['West_SppLo']]  # remove any spaces in species names
for name_old, name_new in spp_chg.items():
    df_vor_june['East_SppLo'] = df_vor_june['East_SppLo'].replace(name_old, name_new)  # replace erroneous species names
    df_vor_june['West_SppLo'] = df_vor_june['West_SppLo'].replace(name_old, name_new)  # replace erroneous species names
df_vor_june['East_SppLo'] = df_vor_june['East_SppLo'].str.upper()  # convert all species names to upper case
df_vor_june['West_SppLo'] = df_vor_june['West_SppLo'].str.upper()  # convert all species names to upper case
df_vor_june.loc[df_vor_june['East_SppLo'].isin(spp_rem), 'East_Low1'] = np.nan
df_vor_june.loc[df_vor_june['East_SppLo'].isin(spp_rem), 'East_High1'] = np.nan
df_vor_june.loc[df_vor_june['West_SppLo'].isin(spp_rem), 'West_Low2'] = np.nan
df_vor_june.loc[df_vor_june['West_SppLo'].isin(spp_rem), 'West_High2'] = np.nan
df_vor_june['Mean_Low'] = np.nanmean(df_vor_june[['East_Low1', 'West_Low2']], axis=1)
df_vor_june['Mean_High'] = np.nanmean(df_vor_june[['East_High1', 'West_High2']], axis=1)
df_vor_june['bm'] = df_vor_june['Mean_Low'] * coef_low + df_vor_june['Mean_High'] * coef_hi + coef_intcp
df_vor_june['Plot'] = "P" + df_vor_june['Plot'].astype('str')

C:\Users\SEAN~1.KEA\AppData\Local\Temp/ipykernel_61624/3626740471.py:26: RuntimeWarning: Mean of empty slice
  df_vor_june['Mean_Low'] = np.nanmean(df_vor_june[['East_Low1', 'West_Low2']], axis=1)
C:\Users\SEAN~1.KEA\AppData\Local\Temp/ipykernel_61624/3626740471.py:27: RuntimeWarning: Mean of empty slice
  df_vor_june['Mean_High'] = np.nanmean(df_vor_june[['East_High1', 'West_High2']], axis=1)


#### Load and prepare October VOR data

In [66]:
df_vor_oct = pd.read_csv(os.path.join(vor_baseDIR, vor_octDIR, vor_oct_f),
                          usecols=np.arange(0, 16), engine='python')
df_vor_oct.loc[df_vor_oct['Pasture'] == 'NH', 'Pasture'] = '10S'
df_vor_oct = df_vor_oct[~df_vor_oct.isnull().all(axis=1)]
df_vor_oct['Date'] = ['10/5/' + str(int(i)) for i in df_vor_oct['Year']]
df_vor_oct['Date'] = pd.to_datetime(df_vor_oct['Date'], format='%m/%d/%Y')
df_vor_oct.loc[df_vor_oct['SamplingDate'].notnull(), 'Date'] = pd.to_datetime(df_vor_oct.loc[df_vor_oct['SamplingDate'].notnull(), 'SamplingDate'])
df_vor_oct['East_SppLo'] = [re.sub(' ', '', str(x)) for x in
                           df_vor_oct['East_SppLo']]  # remove any spaces in species names
df_vor_oct['West_SppLo'] = [re.sub(' ', '', str(x)) for x in
                           df_vor_oct['West_SppLo']]  # remove any spaces in species names
for name_old, name_new in spp_chg.items():
    df_vor_oct['East_SppLo'] = df_vor_oct['East_SppLo'].replace(name_old, name_new)  # replace erroneous species names
    df_vor_oct['West_SppLo'] = df_vor_oct['West_SppLo'].replace(name_old, name_new)  # replace erroneous species names
df_vor_oct['East_SppLo'] = df_vor_oct['East_SppLo'].str.upper()  # convert all species names to upper case
df_vor_oct['West_SppLo'] = df_vor_oct['West_SppLo'].str.upper()  # convert all species names to upper case
df_vor_oct.loc[df_vor_oct['East_SppLo'].isin(spp_rem), 'East_Low1'] = np.nan
df_vor_oct.loc[df_vor_oct['East_SppLo'].isin(spp_rem), 'East_High1'] = np.nan
df_vor_oct.loc[df_vor_oct['West_SppLo'].isin(spp_rem), 'West_Low2'] = np.nan
df_vor_oct.loc[df_vor_oct['West_SppLo'].isin(spp_rem), 'West_High2'] = np.nan
df_vor_oct['Mean_Low'] = np.nanmean(df_vor_oct[['East_Low1', 'West_Low2']], axis=1)
df_vor_oct['Mean_High'] = np.nanmean(df_vor_oct[['East_High1', 'West_High2']], axis=1)
#df_vor_oct['Date'] = [datetime(i, 10, 5) for i in df_vor_oct['Year']]
df_vor_oct['bm'] = df_vor_oct['Mean_Low'] * coef_low + df_vor_oct['Mean_High'] * coef_hi + coef_intcp
df_vor_oct['Plot'] = "P" + df_vor_oct['Plot'].astype('str')

#### Final cleanup, aggregation (to plot scale) and concatenation of VOR data

In [81]:
if exp_unit == "Transect":
    df_vor_june['Id'] = df_vor_june['Pasture'] + "_" + df_vor_june['Plot'] + "_" + df_vor_june['Transect'].astype('str')
    df_vor_oct['Id'] = df_vor_oct['Pasture'] + "_" + df_vor_oct['Plot'] + "_" + df_vor_oct['Transect'].astype('str')
if exp_unit == "Plot":
    df_vor_june['Id'] = df_vor_june['Pasture'] + "_" + df_vor_june['Plot']
    df_vor_oct['Id'] = df_vor_oct['Pasture'] + "_" + df_vor_oct['Plot']
df_vor['Id'] = df_vor['Pasture'] + "_" + df_vor['Plot']

# final cleanup of data
df_vor.loc[df_vor['High'] > max_vor, 'bm'] = np.nan
df_vor.loc[df_vor.High > (
        df_vor.groupby(['Id', 'Date']).High.transform('mean') +
        df_vor.groupby(['Id', 'Date']).High.transform('std') * max_sd), 'bm'] = np.nan
df_vor = df_vor[~(df_vor['bm'].isna())]

df_vor_june = df_vor_june[~(df_vor_june.East_Low1 > df_vor_june.East_High1)]  # remove nonsensical data
df_vor_june = df_vor_june[~(df_vor_june.West_Low2 > df_vor_june.West_High2)]  # remove nonsensical data
df_vor_june.loc[(df_vor_june.East_High1 > max_vor) | (df_vor_june.West_High2 > max_vor), 'bm'] = np.nan
df_vor_june.loc[df_vor_june.Mean_High > (
        df_vor_june.groupby(['Id', 'Date']).Mean_High.transform('mean') +
        df_vor_june.groupby(['Id', 'Date']).Mean_High.transform('std') * max_sd), 'bm'] = np.nan
df_vor_june = df_vor_june[~(df_vor_june['bm'].isna())]

df_vor_oct = df_vor_oct[~(df_vor_oct.East_Low1 > df_vor_oct.East_High1)]  # remove nonsensical data
df_vor_oct = df_vor_oct[~(df_vor_oct.West_Low2 > df_vor_oct.West_High2)]  # remove nonsensical data
df_vor_oct.loc[(df_vor_oct.East_High1 > max_vor) | (df_vor_oct.West_High2 > max_vor), 'bm'] = np.nan
df_vor_oct.loc[df_vor_oct.Mean_High > (
        df_vor_oct.groupby(['Id', 'Date']).Mean_High.transform('mean') +
        df_vor_oct.groupby(['Id', 'Date']).Mean_High.transform('std') * max_sd), 'bm'] = np.nan
df_vor_oct = df_vor_oct[~(df_vor_oct['bm'].isna())]

df_vor_seas = pd.concat([df_vor_june, df_vor_oct])

unique_SPP = pd.DataFrame(dict(species=pd.concat([df_vor_june.East_SppLo,
                                                  df_vor_june.West_SppLo,
                                                  df_vor_oct.East_SppLo,
                                                  df_vor_oct.West_SppLo,
                                                  df_vor.Spp]).unique()))

#### Write data to disk

In [86]:
df_vor_seas.to_csv(os.path.join(outDIR, 'cper_vor_june_oct_2013_2021.csv'), index=False)

df_vor.to_csv(os.path.join(outDIR, 'cper_vor_wkly_2013_2021.csv'), index=False)